# Summary


These are things that are not 100% expected or that I ran into while coding and wanted to never run into again.


# Closures and Variable Capture


## Capture by Value

As you'd expect/hope, the parameter y is not a reference to the variable passed in. I included this not because it's unexpected but because it stands in contrast to what's about to happen.


In [1]:
x = 10


def f(y):
    y = 20


f(x)
x

10

## Closures

You might intuitively expect this to print the numbers 0 to 9, but it prints 9 over and over. The reason is because the lambda **closes over a reference to i**. By the time the lambdas are returned, i has been updated to 9. When it is called after the function, the last value is seen by all the lambdas.


In [3]:
def make_fns(x):
    lambdas = []
    for i in range(x):
        lambdas.append(lambda: print(i))
    return lambdas


fns = make_fns(10)
for fn in fns:
    fn()

9
9
9
9
9
9
9
9
9
9


The same thing happens here. The issue is **not specific to the lambda keyword**.


In [8]:
def make_fns(x):
    fns = []
    for i in range(x):

        def make_fns_interior():
            print(i)

        fns.append(make_fns_interior)
    return fns


fns = make_fns(10)
for fn in fns:
    fn()

9
9
9
9
9
9
9
9
9
9


If you need it to close over a variable by value instead, the solution is to add an extra layer of indirection with an **inline def** like below. Unlike the inline def used above, this one wraps the lambda with a layer that **captures the variable by value**.


In [5]:
def make_fns(x):
    lambdas = []
    for i in range(x):

        def make_fns_lambda(val):
            return lambda: print(val)

        lambdas.append(make_fns_lambda(i))
    return lambdas


fns = make_fns(10)
for fn in fns:
    fn()

0
1
2
3
4
5
6
7
8
9


Here is an example of why closure by reference is useful and not just a burden.


In [19]:
# Assume this is part of the API of a file-like object.
def value_retriever():
    values = [1, 2, 3, 4, 5]
    current_value_index = 0

    def value_retriever_lambda():
        nonlocal current_value_index, values  # required sometimes

        if current_value_index >= len(values):
            return None
        current_value = values[current_value_index]
        current_value_index += 1
        return current_value

    return value_retriever_lambda


# This part is how the client sees it.
reader = value_retriever()
next_val = reader()
while next_val:
    print(next_val)
    next_val = reader()

1
2
3
4
5


In this case, it's ok that it closes by reference because you don't change it within the enclosing scope.


In [22]:
def make_f(x):
    # OK to close over x by ref because we don't touch it
    return lambda val: val + x


f = make_f(10)
f(5)

15

**Summary**: in a def or a lambda, if it's a parameter, it's by value. Otherwise, it's by reference. If it's by value in the enclosing scope, then that protects it from whatever happens in the next higher scope.


## Multi-Statement Lambdas

In other languages like C# and C++, you can use {} to make a multi-statement lambda. In Python, the lambda statement doesn't do this - you have to use def.


In [23]:
def make_f():

    def f():
        print(10)
        print(20)

    return f

# Variable Scope


## Reading Global


In [25]:
x = 1


def f():
    return x


x = 2
f()

2

## Assigning Global


This won't assign it - the assignment is assumed to create a new **shadowing variable** by default.


In [26]:
x = 1


def f():
    x = 2


f()
x

1

A special keyword is needed.


In [30]:
x = 1


def f():
    global x
    x = 2


f()
x

2

## Assigning to Higher Scope

**nonlocal** keyword tells it to look at the next highest scoped version of the variable name.

NOTE: you cannot use nonlocal to refer to a global variable!

NOTE: if the closure was only reading the value, you would not need the nonlocal keyword.


In [34]:
def make_f():
    x = 0

    def f():
        nonlocal x
        x = x + 1
        return x

    return f


f = make_f()
print(f())
print(f())

1
2


## Mutable State

This part is like many other languages. C++ is the main one where this is preventable/controllable.

Even though the variable was captured by value in f(), it is a reference to an object, and that object is a reference. So it is mutable.

The only way to prevent that is to use a special immutable version of the class (as in Java).


In [47]:
def f(somedict):
    somedict['x'] = 'new'


x = {'x': 'old'}
f(x)

print(x)

{'x': 'new'}


# Static vs. Instance Variables

This part works significantly **differently from other languages**.

The big place this bit me is in **unit tests** since you get a new class instance for every test method (which is not hermetically sealed if you accidentally assigned static data).

Summary:

- When you create an instance, the current static value (if it exists) is **copied** into the instance.
- Assigning through `self` or externally via the member variable only affects the **instance**.
- Instances are independent, but future new instances are affected by changes to the static version.


In [46]:
class MyClass:
    x = 1

    def f(self):
        self.x = 10


print(f'original value (static): {MyClass.x}')

MyClass.x = 2
print(f'static after reassign: {MyClass.x}')

m = MyClass()
print(f'instance variable after construction: {m.x}')

m.f()
print(f'instance variable after calling method to reassign: {m.x}')
print(f'static variable after same method call: {MyClass.x}')

m.x = 5
print(f'static variable after reassigning instance: {MyClass.x}')

n = MyClass()
print(f'new instance after construction: {n.x}')

print(f'final state: MyClass.x={MyClass.x}, m.x={m.x}, n.x={n.x}')

original value (static): 1
static after reassign: 2
instance variable after construction: 2
instance variable after calling method to reassign: 10
static variable after same method call: 2
static variable after reassigning instance: 2
new instance after construction: 2
final state: MyClass.x=2, m.x=5, n.x=2


This is the canonical way to make an instance variable that is not also a static variable.


In [48]:
class MyClass:

    def __init__(self):
        self.x = 5

# Constructors


If a derived class has no constructor, the base constructor is called.


In [49]:
class MyBase:

    def __init__(self):
        self.x = 5


class MyClass(MyBase):
    pass


m = MyClass()
print(m.x)

5


Here's the weird part - unlike other languages - if the subclass provides a constructor, then the **base is not called** anymore!


In [51]:
class MyClass(MyBase):

    def __init__(self):
        self.y = 10


m = MyClass()
try:
    print(m.x)
except AttributeError:
    print('fail!')

fail!


The fix is to **call the base** class constructor like this. Basically any time you subclass, you need to do this.


In [53]:
class MyClass(MyBase):

    def __init__(self):
        super().__init__()
        self.y = 10


m = MyClass()
print(m.x)

5


## Member Variables Referencing Each Other


Static variables can reference each other without having to use the classname in front.


In [54]:
class MyClass:
    x = 5
    y = x


m = MyClass()
print(m.x)
print(m.y)

5
5


# Variable Lifetime


Variables created by with statements **outlive the with statement** but are in a closed state.


In [58]:
class MyContext:

    def __enter__(self):
        return 10

    def __exit__(self, exc_type, exc_val, exc_tb):
        print('closed')


with MyContext() as x:
    print(x)
print(x)

10
closed
10


This is true for **other block constructs** as well.


In [60]:
for i_from_loop in range(10):
    pass
print(i_from_loop)

9


It is not limited to the declaration part of the block.


In [61]:
for i in range(10):
    j = i**2
print(j)

81


The benefit of this is the ability to do something like this:


In [63]:
def f(x):
    if x < 0:
        y = 0  # y declared via branches without needing pre-declaration
    else:
        y = x
    return y**2

# Bound vs. Unbound Methods


In [71]:
class MyClass:

    def f(self, x):
        print(self)
        print(x)
        return x**2


m = MyClass()

print('((Bound))')
# self is automatically passed to MyClass.f in lambda-like usage.
print(list(map(m.f, [1, 2, 3])))

print()
print('((Unbound))')
# using it statically, it is unbound, so you have to pass a self.
print(list(map(MyClass.f, [None, None, None], [1, 2, 3])))

((Bound))
1
2
3
[1, 4, 9]

((Unbound))
None
1
None
2
None
3
[1, 4, 9]


Note that **f and g are equivalent** below and thus you can avoid some extra syntax.


In [72]:
f = lambda x: m.f(x)
g = m.f

f(10)
g(10)

10
10


100

# Tuple Comprehensions

Because () without tuple is already taken for **generator comprehensions**, you have to **cast to tuple** if you want that.


In [74]:
t = tuple(i**2 for i in range(3))
print(t)

(0, 1, 4)


# Empty Set Literal

You can't use {} for set literal because it already means dict literal. You have to use the explicit constructor.


In [75]:
print(type({}))
print(type(set()))

<class 'dict'>
<class 'set'>


# Dictionary Members

This doesn't usually matter, but in some cases, such as passing to functions that explicitly expect lists or tuples, it might matter. Dictionary keys and values members are not explicitly lists or tuples until you **cast them**.


In [81]:
d = {'a': 1, 'b': 2}
print(isinstance(d.keys(), list))
print(isinstance(d.keys(), tuple))
print(isinstance(d.values(), list))
print(isinstance(d.values(), tuple))

print(list(d.keys()))

False
False
False
False
['a', 'b']


# Formatting of Fluent APIs


This is what it will look like if you leave it to the formatter by default.


In [83]:
class MyClass:

    def __init__(self):
        self.x = 1

    def map(self, x):
        self.x = x
        return self


m = MyClass().map(10).map(20).map(30).map(40).map(50).map(60).map(70).map(
    80).map(90).map(100)


You can protect it from the formatter by using \ to continue lines before they get too long.


In [84]:
m = MyClass().map(10).map(20).map(30)\
             .map(40).map(50).map(60)\
             .map(70).map(80).map(90).map(100)

# Multiline Statements


This is an **error** because python had no signal that this was going to continue on the next line.


In [89]:
class MyClass:

    def __init__(self):
        self.x = 1

    def map(self, x):
        self.x = x
        return self

m = MyClass().map(10)
             .map(10)

IndentationError: unexpected indent (2404800938.py, line 11)

You can use \ or (), {}, or [] to signal a line continuation, and then the indentation is up to you.


In [91]:
m = (
    MyClass()  # comment here to stop formatter from rejoining
    .map(10))
n = MyClass()\
        .map(10)

# Equality Operators


In [94]:
print([1, 2, 3] == [1, 2, 3])  # value equal
print([1, 2, 3] is [1, 2, 3])  # reference equal


True
False


In [98]:
print([1, 2, 3] is not [1, 2, 3])  # note that it's 'is not' instead of 'not is'


True


In [100]:
x = None
print(x is None)
print(x == None)  # this works too, but the linter will yell at you


True
True


# Boolean


All the usual suspects for **falsey** values are boolean false.


In [107]:
x = None
if x:
    print(True)
else:
    print(False)

y = False
if y:
    print(True)
else:
    print(False)

y = 0
if y:
    print(True)
else:
    print(False)

y = ''
if y:
    print(True)
else:
    print(False)

y = []
if y:
    print(True)
else:
    print(False)

print(not True)
print(False)

False
False
False
False
False
False
False


In [110]:
x = []
if x is not None:
    print(bool(x))  # disambiguating None from empty list

False


Word operators like **and, or, not** convert operands to boolean and return pure boolean.


In [128]:
x = []
y = None
if x or y:
    print('x or y')  # not printed
print(y or not x)  # printed


True


Bitwise operators **&, |, ~**. There is no such thing as &&, || here like in other languages (those are the words above instead).

Some APIs override these for other purposes such as combining sets, etc.


In [134]:
print(~1)
print(1 & 3)
print(1 | 2)

-2
1
3


# Declaration Order

Within bodies of classes and functions, you are ok to reference symbols that haven't happened yet. They won't be resolved until the thing is called.


In [2]:
class AClass:

    def f():
        return BClass()


class BClass:

    def f():
        return AClass()